### File used to generate unforced trajectories, for comparison between learned and true dynamics

In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
	sys.path.insert(0, module_path)
print(sys.path)

import Double_Pendulum.Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Starting parameters

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS.copy()
#rp["m0"] = 0.
plotter = pendulum_plot.Anim_plotter(rp)
neural_net = False
model_cw = False
model_shifting = True
print(rp["m0"])
print(rp)

armlen = rp["l0"] + rp["l1"]

q0_max = 0.

In [ ]:
t_start = 0.
t_end = 3.
dt = 0.01

model = autoencoders.Autoencoder_double(rp).to(device)
model_location = 'Models/NN_202505141642(half-q)/NN_202505141642_0.pth'
model.load_state_dict(torch.load(model_location, weights_only=True))

In [ ]:
def ik_fn(xy):
	return transforms.inverse_kinematics(xy, rp, model_cw).squeeze(0)

#### Generate $q$-space trajectory

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
def sim_q(t_start, t_end, dt, q_start, q_d_start, u):
	q = q_start
	q_d = q_d_start
	qs_ana = []
	q_ds_ana = []


	n_steps = int((t_end - t_start) / dt)

	for _ in range(n_steps):
		
		M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q = dynamics.input_matrix(rp, q.squeeze(0))

		tau_q = A_q * u
		q_dd = (torch.pinverse(M_q) @ (tau_q - C_q @ q_d.T - G_q)).T
		q_d = q_d + q_dd * dt
		q = q + q_d * dt
		
		qs_ana.append(q.squeeze(0).detach())
		q_ds_ana.append(q_d.squeeze(0).detach())
	
	return torch.stack(qs_ana,  dim=0), torch.stack(q_ds_ana, dim=0)

In [ ]:
def sim_q_old(t_start, t_end, dt, q_start, q_d_start, u):
	q = q_start
	q_d = q_d_start

	q_ana_series, q_d_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	n_steps = int((t_end - t_start) / dt)

	for t in torch.arange(t_start, t_end, dt):

		M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q = dynamics.input_matrix(rp, q.squeeze(0))

		tau_q = A_q * u
		q_dd = (torch.pinverse(M_q) @ (tau_q - C_q @ q_d.T - G_q)).T
		q_d = q_d + q_dd * dt
		q = q + q_d * dt

		q_ana_series = torch.cat((q_ana_series, q.detach()), dim=0)
		q_d_ana_series = torch.cat((q_d_ana_series, q_d.detach()), dim=0)

	
	return q_ana_series, q_d_ana_series

#### Generate $\hat{q}$-space trajectory

In [ ]:
timestamp_alt = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

def sim_q_hat(t_start, t_end, dt, q_start, q_d_start, u, model):
	q_nn = q_start
	q_d_nn = q_d_start

	th = model.encoder_vmap(q_nn)
	th_d = (model.jacobian_enc(q_nn) @ q_d_nn.T).T

	q_nn_series, q_d_nn_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	for t in torch.arange(t_start, t_end, dt):

		

		is_clockwise = transforms.check_clockwise(q_nn.squeeze(0))

		if model_shifting:
			q_nn = transforms.shift_q(q_nn, clockwise=model_cw)


		
		q_hat = model.decoder_vmap(th, clockwise=model_cw)
		q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
		

		M_q_hat, C_q_hat, G_q_hat = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
		A_q_hat = dynamics.input_matrix(rp, q_hat.squeeze(0))
		J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
		J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)
		M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_hat, C_q_hat, G_q_hat, th, th_d, J_h_inv, J_h_inv_trans)
		
		A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_hat, J_h_inv_trans)
		
		tau_th = A_th * u
		th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T - G_th)).T
		th_d = th_d + th_dd * dt
		th = th + th_d * dt
		

		
		q_nn = model.decoder_vmap(th)
		q_d_nn = (model.jacobian_dec(th) @ th_d.T).T
		

		q_nn_series = torch.cat((q_nn_series, q_nn.detach()), dim = 0)
		q_d_nn_series = torch.cat((q_d_nn_series, q_d_nn.detach()), dim = 0)

	return q_nn_series, q_d_nn_series


In [ ]:
def make_xy_series(q_ana_series, q_nn_series):

	xy_ana_series = torch.empty((0,2)).to(device)
	xy_nn_series = torch.empty((0,2)).to(device)

	for q_ana, q_nn in zip(q_ana_series, q_nn_series):
		xy_ana, _ = transforms.forward_kinematics(rp, q_ana)
		xy_ana_series = torch.cat((xy_ana_series, xy_ana.unsqueeze(0)))
		xy_nn, _ = transforms.forward_kinematics(rp, q_nn)
		xy_nn_series = torch.cat((xy_nn_series, xy_nn.unsqueeze(0)))
	
	return xy_ana_series, xy_nn_series

In [ ]:
def check_stretched_arm_index(xy_series, arm_length):
	range_series = torch.sqrt(torch.sum(torch.square(xy_series), dim=1))
	too_long = range_series > 0.95 * arm_length
	if too_long.any():
		idxs = torch.nonzero(too_long, as_tuple=True)[0]
		too_long_idx = idxs[0].item()
		return too_long_idx
	else:
		return xy_series.size(0)
	
def check_q_outside_index(q_series, q0_max):
	outside = (q_series[:, 0] > q0_max) | (q_series[:, 0] < -torch.pi)

	if outside.any():
		idxs = torch.nonzero(outside, as_tuple=True)[0]
		outside_idx = idxs[0].item()
		return outside_idx
	else:
		return q_series.size(0)
	
def check_origin_close(xy_series, arm_length):
	range_series = torch.sqrt(torch.sum(torch.square(xy_series), dim=1))
	too_short = range_series < 0.1 * arm_length
	if too_short.any():
		idxs = torch.nonzero(too_short, as_tuple=True)[0]
		too_long_idx = idxs[0].item()
		return too_long_idx
	else:
		return xy_series.size(0)

In [ ]:
def create_data_lists(qs_start, q_ds_start, us, loaded_data = None, print_iter = None, save_path = None):

	arm_length = rp["l0"] + rp["l1"]

	if loaded_data is None:
		q_ana_series_list, q_d_ana_series_list, xy_ana_series_list = [], [], []
		q_nn_series_list, q_d_nn_series_list, xy_nn_series_list = [], [], []
	else:
		q_ana_series_list, q_d_ana_series_list, xy_ana_series_list = loaded_data["q_ana_sl"], loaded_data["q_d_ana_sl"], loaded_data["xy_ana_sl"]
		q_nn_series_list, q_d_nn_series_list, xy_nn_series_list = loaded_data["q_nn_sl"], loaded_data["q_d_nn_sl"], loaded_data["xy_nn_sl"]


	for i, (q_start, q_d_start, u) in enumerate(zip(qs_start, q_ds_start, us)):
		if print_iter is not None:
			if i % print_iter == 0:
				print("sim nr:", i)
		q_ana_series, q_d_ana_series = sim_q(t_start, t_end, dt, q_start.clone(), q_d_start.clone(), u)
		q_nn_series, q_d_nn_series = sim_q_hat(t_start, t_end, dt, q_start.clone(), q_d_start.clone(), u, model)
		

		xy_ana_series, xy_nn_series = make_xy_series(q_ana_series, q_nn_series)

		too_long_idx = check_stretched_arm_index(xy_ana_series, arm_length)
		q_outside_idx = check_q_outside_index(q_ana_series, q0_max)
		too_short_idx = check_origin_close(xy_ana_series, arm_length)
		save_idx = min(too_long_idx, q_outside_idx, too_short_idx)
		
		if save_idx > 0:
			q_ana_series_list.append(q_ana_series[:save_idx])
			q_d_ana_series_list.append(q_d_ana_series[:save_idx])
			xy_ana_series_list.append(xy_ana_series[:save_idx])

			q_nn_series_list.append(q_nn_series[:save_idx])
			q_d_nn_series_list.append(q_d_nn_series[:save_idx])
			xy_nn_series_list.append(xy_nn_series[:save_idx])

		if save_path is not None:
			data_to_save = {
				"q_ana_sl": q_ana_series_list,
				"q_d_ana_sl": q_d_ana_series_list,
				"xy_ana_sl": xy_ana_series_list,
				"q_nn_sl": q_nn_series_list,
				"q_d_nn_sl": q_d_nn_series_list,
				"xy_nn_sl": xy_nn_series_list,
			}
			torch.save(data_to_save, save_path)
	
	return q_ana_series_list, q_d_ana_series_list, xy_ana_series_list, q_nn_series_list, q_d_nn_series_list, xy_nn_series_list

In [ ]:
def save_metadata(xy_start):
	save_dir = f"Performance_Sims/simulation_{timestamp}"
	os.makedirs(save_dir, exist_ok=True)

	metadata = {
		"timestamp": timestamp,
		"Learned transform": False,
		"Model location": None,
		"Model clockwise": None,
		"Model shifting": None, 
		"xy start": {"x": xy_start[0].item(), "y": xy_start[1].item()},
		"Time step": dt,
		"Sim time": t_end,
	}

	metadata_path = os.path.join(save_dir, "metadata.json")
	with open(metadata_path, "w") as f:
		json.dump(metadata, f, indent=4)

In [ ]:
def make_anim(q_ana_series, q_nn_series, save_dir):
	stride = 3
	pos_end_q_ana, pos_elbow_q_ana = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_ana_series[::stride])
	pos_end_q_nn, pos_elbow_q_nn = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_nn_series[::stride])

	frames_q_ana = plotter.frame_pendulum(pos_end_q_ana, pos_elbow_q_ana)
	frames_q_nn = plotter.frame_pendulum(pos_end_q_nn, pos_elbow_q_nn)

	data_ana = {
		"frames": frames_q_ana,
		"times": dt,
		"name": "q_ana", 
		"arm_color": "tab:blue",
		"act_color": "tab:cyan"
	}

	data_nn = {
		"frames": frames_q_nn,
		"times": dt,
		"name": "q_est",
		"arm_color": "tab:orange", 
		"act_color": "tab:red"
	} 


	frames_data = [data_ana, data_nn]

	name_rp = "RP:(" + str(rp["xa"]) + "," + str(rp["ya"]) + ")_"


	if neural_net:
		model_type = "NN"
	else:
		model_type = "AL"

	file_name = "t_end:[" + str(t_end) + "]_dt:[" + str(dt) + "]_stride:[" + str(stride) + "]_0.mp4"
	#file_counter = 0

	output_path = os.path.join(save_dir, file_name)

	while os.path.isfile(output_path):
		print("file name already exists")
		file_counter += 1
		file_name = file_name[:-6] + "_" + str(file_counter) + ".mp4"
		output_path = os.path.join(save_dir, file_name)

	#plotter.animate_pendulum(frames_data, ref_poss=None, plot_actuator=False, save_path=output_path, fps = 1/(dt*stride), dt = dt*stride)


In [ ]:
def create_plot_datasets(q_ana_series, q_nn_series, xy_ana_series, xy_nn_series):
	datasets_q = [
		{
			"name": "Learned",
			"values": q_nn_series.cpu().detach().numpy(),
			"color": "tab:orange"
		},
		{
			"name": "Ground truth",
			"values": q_ana_series.cpu().detach().numpy(),
			"color": "tab:blue"
		}
	]

	datasets_xy = [
		{
			"name": "Learned",
			"values": xy_nn_series.cpu().detach().numpy(),
			"color": "tab:orange"
		},
		{
			"name": "Ground truth",
			"values": xy_ana_series.cpu().detach().numpy(),
			"color": "tab:blue"
		}
	]


	# Common labels for the plots.
	name_q = "q trajectory"
	name_xy = "xy trajectory"
	t_series = torch.arange(0, t_end, dt)
	t_series = t_series[:q_ana_series.size(0)]

	# Create an instance of ErrorPlotter.
	ep = pendulum_plot.Error_plotter(rp)

	# Prepare plot datasets for each column.
	# Each call groups a set of datasets to be drawn in one subplot column.
	q_plot_dataset = ep.create_plot_dataset(t=t_series, datasets=datasets_q, reference=None, name=name_q)
	xy_plot_dataset = ep.create_plot_dataset(t=t_series, datasets=datasets_xy, reference=None, name=name_xy)


	plot_datasets = [q_plot_dataset, xy_plot_dataset]
	plot_colormaps = ["Oranges", "Blues", "Greens"]

	return plot_datasets, plot_colormaps

In [ ]:

def make_error_plots(plot_datasets, save_dir, ep):
	file_name = "Error plot.png"
	file_counter = 0

	output_path = os.path.join(save_dir, file_name)

	# Pass the list of columns (plot_dataset objects) to plot_multi.
	ep.plot_multi(plot_datasets=plot_datasets, save_path=output_path, axes_names = ["q", "th", "xy"])




#### Plot $q$-space error

In [ ]:
from matplotlib.lines import Line2D
def plot_q_trajs(q_dataset, plot_colormaps):
	fig, ax = plt.subplots(figsize=(6,4))
	scs = []
	for i, cmap in zip(range(2), plot_colormaps):
		t = q_dataset["x"]                      # shape (N,)
		q1 = q_dataset["data"][i]["y1"]         # shape (N,)
		q2 = q_dataset["data"][i]["y2"]         # shape (N,)
		sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

		# optionally connect points in order
		ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)
		scs.append(sc)

	# labels and colorbar
	ax.set_xlabel('$q_0$')
	ax.set_ylabel('$q_1$')
	ax.grid()

	cbar1 = fig.colorbar(
		scs[0], ax=ax, pad=-0.01, fraction=0.057
	)
	cbar1.set_label('time')

	cbar2 = fig.colorbar(
		scs[1], ax=ax, pad=0.02, fraction=0.057, ticks=[]
	)

	labels = [q_dataset["data"][i]["name"] for i in range(2)]
	proxies = []
	for cmap, label in zip(plot_colormaps, labels):
		color = plt.get_cmap(cmap)(0.7)
		proxies.append(Line2D([0], [0], marker='o', 
							  color='w',
							  markerfacecolor=color,
							  markersize=8, 
							  linestyle='',
							  label=label))

	ax.legend(handles=proxies, title="Trajectories")

	ax.set_title('Trajectory in $(q_0,q_1)$ colored by time')
	plt.tight_layout()
	plt.show()

In [ ]:
def plot_xy_trajs(xy_dataset, plot_colormaps):
	fig, ax = plt.subplots(figsize=(5,4))
	scs = []
	for i, cmap in zip(range(2), plot_colormaps):
		t = xy_dataset["x"]
		q1 = xy_dataset["data"][i]["y1"]
		q2 = xy_dataset["data"][i]["y2"]
		sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

		# optionally connect points in order
		ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)
		scs.append(sc)

	circ = plt.Circle((0, 0), armlen, color="gray", fill=False, linestyle="dashed", alpha=0.8)
	ax.add_patch(circ)

	# labels and colorbar
	ax.set_xlabel('$x$')
	ax.set_ylabel('$y$')
	ax.grid()

	ax.set_xlim(-armlen, armlen)
	ax.set_ylim(-armlen, armlen)

	cbar1 = fig.colorbar(
		scs[0], ax=ax, pad=-0.0, fraction=0.057
	)
	cbar1.set_label('time')

	cbar2 = fig.colorbar(
		scs[1], ax=ax, pad=-0.14, fraction=0.057, ticks=[]
	)

	labels = [xy_dataset["data"][i]["name"] for i in range(2)]
	proxies = []
	for cmap, label in zip(plot_colormaps, labels):
		color = plt.get_cmap(cmap)(0.7)
		proxies.append(Line2D([0], [0], marker='o', 
							  color='w',
							  markerfacecolor=color,
							  markersize=8, 
							  linestyle='',
							  label=label))

	ax.legend(handles=proxies, title="Trajectories")

	ax.set_title('Trajectory in $(x,y)$ colored by time')
	plt.axis('square')
	plt.tight_layout()
	plt.show()

In [ ]:
n_sims = 1000
n_tries = 4 * n_sims
print_iter = max(torch.floor(torch.tensor(n_sims)/100).item(), 1)

#sample_angle = 2 * torch.pi * torch.rand(n_tries)
#sample_radius = 0.6 * armlen * torch.sqrt(torch.rand(n_tries)) + 0.2 * armlen
#xs = sample_radius * torch.cos(sample_angle)
#ys = sample_radius * torch.sin(sample_angle)

xmin, xmax = 1, 3
ymin, ymax = -1, 1
xs = torch.rand(n_tries) * (xmax - xmin) + xmin
ys = torch.rand(n_tries) * (ymax - ymin) + ymin
xys_start = torch.stack((xs, ys), dim=1).requires_grad_().to(device)


x_dmin, x_dmax = -3, 3
y_dmin, y_dmax = -0.5, 0.5
x_ds = torch.rand(n_tries) * (x_dmax - x_dmin) + x_dmin
y_ds = torch.rand(n_tries) * (y_dmax - y_dmin) + y_dmin
xy_ds_start = torch.stack((x_ds, y_ds), dim=1).requires_grad_().to(device)

qs_start = torch.empty((0,2)).requires_grad_().to(device)
q_ds_start = torch.empty((0,2)).requires_grad_().to(device)

for xy_start, xy_d_start in zip(xys_start, xy_ds_start):
	q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
	q_d_start = (torch.autograd.functional.jacobian(ik_fn, xy_start) @ xy_d_start).unsqueeze(0)

	qs_start = torch.cat((qs_start, q_start))
	q_ds_start = torch.cat((q_ds_start, q_d_start))

keep_points = qs_start[:, 0] < q0_max

xys_start = xys_start[keep_points][:n_sims]
xy_ds_start = xy_ds_start[keep_points][:n_sims]
qs_start = qs_start[keep_points][:n_sims]
q_ds_start = q_ds_start[keep_points][:n_sims]
qs_start = qs_start.unsqueeze(1)
q_ds_start = q_ds_start.unsqueeze(1)

us = -25 - 10 * torch.rand(n_sims)

In [ ]:
load_data = True
if load_data:
	loaded_data = torch.load("Performance_Sims/saved_data_small_xy.pt")

	q_ana_sl = loaded_data["q_ana_sl"]
	q_d_ana_sl = loaded_data["q_d_ana_sl"]
	xy_ana_sl = loaded_data["xy_ana_sl"]
	q_nn_sl = loaded_data["q_nn_sl"]
	q_d_nn_sl = loaded_data["q_d_nn_sl"]
	xy_nn_sl = loaded_data["xy_nn_sl"]

In [ ]:
save_path = "Performance_Sims/saved_data_small_xy_alt.pt"
q_ana_sl, q_d_ana_sl, xy_ana_sl, q_nn_sl, q_d_nn_sl, xy_nn_sl = create_data_lists(qs_start, q_ds_start, us, loaded_data = None, 
																				  print_iter = print_iter, save_path=save_path)

In [ ]:
data_to_save = {
	"q_ana_sl": q_ana_sl,
	"q_d_ana_sl": q_d_ana_sl,	
	"xy_ana_sl": xy_ana_sl,
	"q_nn_sl": q_nn_sl,
	"q_d_nn_sl": q_d_nn_sl,
	"xy_nn_sl": xy_nn_sl,
}

#torch.save(data_to_save, "Performance_Sims/saved_data_lists_full_range.pt")

In [ ]:
q_error_list = []
xy_error_list = []

for i, (qas, qdas, xyas, qns, qdns, xyns) in enumerate(zip(q_ana_sl, q_d_ana_sl, xy_ana_sl, q_nn_sl, q_d_nn_sl, xy_nn_sl)):

	q_error_series = qas - qns
	q_d_error_series = qdas - qdns
	xy_error_series = xyas - xyns
	plot_datasets, plot_colormaps = create_plot_datasets(qas, qns, xyas, xyns)
	q_plot_dataset = plot_datasets[0]
	xy_plot_dataset = plot_datasets[1]
	if qas.size(0) == 300 and i%100 == 0:
		
		plot_q_trajs(q_plot_dataset, plot_colormaps)
		plot_xy_trajs(xy_plot_dataset, plot_colormaps)

# labels and colorbar
ax.set_xlabel('$q_0$')
ax.set_ylabel('$q_1$')
ax.grid()
#ax.set_xlim(-torch.pi, torch.pi)
#ax.set_ylim(-torch.pi, torch.pi)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('time')

ax.set_title('Trajectory in $(q_0,q_1)$ colored by time')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
for i, cmap in zip(range(2), plot_colormaps):
	t = xy_dataset["x"]                      # shape (N,)
	q1 = xy_dataset["data"][i]["y1"]         # shape (N,)
	q2 = xy_dataset["data"][i]["y2"]         # shape (N,)
	sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

	# optionally connect points in order
	ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)

armlen = rp["l0"] + rp["l1"]
circ = plt.Circle((0, 0), armlen, color="gray", fill=False, linestyle="dashed", alpha=0.8)
ax.add_patch(circ)
# labels and colorbar
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid()
ax.set_xlim(-armlen, armlen)
ax.set_ylim(-armlen, armlen)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('time')

ax.set_title('Trajectory in $(x,y)$ colored by time')
plt.tight_layout()
plt.show()